# DICOM RT Tool Illustrative Example with Open-Access Data

Last edited by Kareem Wahid on January 4, 2020.

This notebook demonstrates the various functions and utilities avaliable in the Dicom RT tool package (https://github.com/brianmanderson/Dicom_RT_and_Images_to_Mask). It serves as supplementary information for the Technical Paper titled: "Simple Python Module for Conversions between DICOM Images and Radiation Therapy Structures, Masks, and Prediction Arrays". This notebook works through an example of publically avalaible brain tumor data of T1-w/FLAIR MRI sequences and corresponding RT structure files with multiple segemented regions of interest. Full information of the publically avaliable brain tumor data used in this notebook can be found at: https://figshare.com/articles/dataset/Data_from_An_Investigation_of_Machine_Learning_Methods_in_Delta-radiomics_Feature_Analysis/9943334. This notebook was written for easy accesibility for beginners to Python programming, medical imaging, and computational analyis.

The notebook covers the following topics (click to go to section):
1. [Getting the Data](#DATA)
2. [Reading in DICOM and RT struct files and converting to numpy array format](#DICOM)
3. [Saving arrays to nifti format and reloading them](#NIFTI)
4. [Saving and loading numpy files for later use](#NUMPY)
5. [Calculating radiomic features from converted numpy arrays](#RADIOMICS)

The notebook assumes you have the following nested directory structure after running cells that download neccessary data:

In [2]:
# importing neccessary libraries 

# file mangagment 
import os 
import zipfile
from six.moves import urllib

# array manipulation and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# medical image manipulation 
import SimpleITK as sitk
from DicomRTTool.ReaderWriter import DicomReaderWriter # pip install DicomRTTool, uncomment this when PyPI is updated
from radiomics import featureextractor # pip install pyradiomics

ModuleNotFoundError: No module named 'radiomics'

In [ ]:
os.chdir("..") # move up one folder to access src folder, temporary solution until PyPI is updated
cwd = os.getcwd()

In [ ]:
from src.DicomRTTool.ReaderWriter import DicomReaderWriter 

In [ ]:
os.chdir("Examples") # change working directory back to what it was 
cwd = os.getcwd()

## Part 1: Getting the Data <a name="DATA"></a>

The RT struc files and their corresponding DICOM images can be in the same directory or different directories. Here we show a case where files are located in the same directory. This is a good dataset to work with since its somwhat messy but coherent enough to show power of DICOMRTTool. Many files but only pre-RT T1 and FLAIR images have assocaited RT structure files. Downloading and unzipping the neccessary files will take about 10 minutes on most CPUs and takes up about 8 GB of storage. One may visalize these DICOM images using a free commericially avaliable DICOM viewer, such as Radiant (https://www.radiantviewer.com/).

In [5]:
%%time
data_path = os.path.join('.', 'Example_Data')
if not os.path.isdir(data_path): # create Example_data directory if it doesn't exist
    os.mkdir(data_path)

url_img = "https://ndownloader.figshare.com/files/20140100" # brain scans 
filename_img = os.path.join(data_path, 'Data.zip')
if not os.path.exists(filename_img): # if zip file doesnt exist download 
    print ("Retrieving zipped images...")
    print('Estimated dowload time is 5 minutes...')
    urllib.request.urlretrieve(url_img, filename_img)
    print('Finished downloading!')
else:
    print ("Zipped images already downloaded.")

if os.path.exists(filename_img):  # If we downloaded the data
    if not os.path.exists(os.path.join(data_path, 'Image_Data')): # and it hasn't been unzipped
        print ("Unzipping images...")
        print('Estimated unzip time is 2 minutes')
        z = zipfile.ZipFile(filename_img)
        z.extractall(data_path)
        print ("Done unzipping images.")
    
print("All required files downloaded and unzipped!") # print when done

Zipped images already downloaded.
All required files downloaded!
Wall time: 1.97 ms


In [6]:
def display_slices(image, mask):
    """
    Displays a series of slices that contains the segmented regions of interest.
    Ensures all contours are displayed in consistent and different colors.
        Parameters:
            image (array-like): numpy array of image 
            mask (array-like): numpy array of mask
        Returns:
            None (series of in-line plots)
    """

    slice_locations = np.unique(np.where(mask != 0)[0]) # get indexes for where there is a contour present 
    slice_start = slice_locations[0] # first slice of contour 
    slice_end = slice_locations[len(slice_locations)-1] # last slice of contour
    
    for img_arr, contour_arr in zip(image[slice_start:slice_end+1], mask[slice_start:slice_end+1]): # plot the slices with contours overlayed ontop
        masked_contour_arr = np.ma.masked_where(contour_arr == 0, contour_arr)
        plt.imshow(img_arr, cmap='gray', interpolation='none')
        plt.imshow(masked_contour_arr, cmap='cool', interpolation='none', alpha=0.5, vmin = 1, vmax = np.amax(mask)) # vmax is set as total number of contours so same colors can be displayed for each slice
        plt.show()

## Part 2: Reading in DICOM and RT struct files and converting to numpy array format. <a name="DICOM"></a>
The principal on which these set of tools operates on is based on the DicomReaderWriter object. It is instantiated with the contours of interest (and associations) and can then be used to create numpy arrays of images and masks of the format [slices, width, height].

To extrapolate the following code logic to a arbitrary folder structure, one could use an os.walk through directories of interest. For example, I normally use a folder structure MRN -> date of image (pre,mid,post-RT) -> type of scan (MRI, CT, etc.) -> files.

In [9]:
DICOM_path = os.path.join('.', 'Example_Data', 'Image_Data') # folder where downloaded data was stored
print(DICOM_path)

.\Example_Data\Image_Data


In [ ]:
%%time
Dicom_reader = DicomReaderWriter(description='Examples', arg_max=True)
Dicom_reader.walk_through_folders(DICOM_path) # need to define in order to use all_roi method

In [ ]:
all_rois = Dicom_reader.return_rois(print_rois=True)  # Return a list of all rois present, and print them

In [ ]:
Contour_Names = ['brainstem', 'abv', 'gtvplus2'] 
associations = {'brainstem1':'brainstem', # just to show an example of how associations works
               'abv_roi': 'abv'}

In [ ]:
Dicom_reader.set_contour_names(Contour_Names)
Dicom_reader.__set_associations__(associations)

Note: Printing "Found []" because many of the scans (post 1 and post 2 RT) do not have assocaited structure files.

In [ ]:
Dicom_reader.set_index(11)  # Setting to 11 since this index has all the structures, corresponds to T1-w image for patient 12
Dicom_reader.get_images_and_mask()  # Load up the images and mask for the requested index

In [ ]:
image = Dicom_reader.ArrayDicom # image array
mask = Dicom_reader.mask # mask array

In [ ]:
display_slices(image, mask) # visualize that our segmentations were succesfully convereted 

Note: Cyan denotes brainstem, magenta denotes tumor, lavender denotes abv (unknown what this ROI actually represents..)

## Part 3: Saving arrays to nifti format. <a name="NIFTI"></a>

If you want to use a manual approach, you can view the nifti files easily after running get_images_and_mask()

In [ ]:
nifti_path = os.path.join('.', 'nifti') # go into nifti subfolder 
if not os.path.exists(nifti_path):
    os.makedirs(nifti_path)

In [ ]:
dicom_sitk_handle = Dicom_reader.dicom_handle
mask_sitk_handle = Dicom_reader.annotation_handle
sitk.WriteImage(dicom_sitk_handle, os.path.join(nifti_path, 'Image.nii'))
sitk.WriteImage(mask_sitk_handle, os.path.join(nifti_path, 'Mask.nii'))

Note the nifti writer creates a log .txt file in each DICOM folder and a corresponding log excel file in the specified output path.

In [ ]:
%%time
Dicom_reader.write_parallel(out_path = nifti_path, excel_file = os.path.join(nifti_path,'.','MRN_Path_To_Iteration.xlsx'))

In [ ]:
nifti_image = sitk.ReadImage(os.path.join(nifti_path,"Overall_Data_Examples_0.0.nii.gz")) # reload image
image = np.array(nifti_image.dataobj)
image = np.moveaxis(image,-1,0) # have to swap axis because nyul scripts orient patient differently 
image = np.swapaxes(image, 1, 2)
nifti_mask = sitk.ReadImage(os.path.join(nifti_path,"Overall_mask_Examples_y0.0.nii.gz")) # reload mask
mask = np.array(nifti_mask.dataobj)
mask = np.moveaxis(mask,-1,0) # have to swap axis because nyul scripts orient patient differently 
mask = np.swapaxes(mask, 1, 2)

In [ ]:
display_slices(image, mask) # visualize that our segmentations were succesfully convereted from nifti 

## Part 4: Saving and loading numpy files for later use. <a name="NUMPY"></a>

Finally we can save the numpy arrays to files for later use (so you don't have to reinstantiate the computationally expensive DicomReaderWriter object) and subsequently re-load the numpy arrays.

In [ ]:
numpy_path = os.path.join('.', 'numpy') # go into numpy subfolder 
if not os.path.exists(numpy_path):
    os.makedirs(numpy_path)

In [ ]:
np.save(os.path.join(numpy_path,'image'), image) # save the arrays
np.save(os.path.join(numpy_path,'mask'), mask)

In [ ]:
image = np.load(os.path.join(numpy_path,'image.npy')) # load the arrays
mask = np.load(os.path.join(numpy_path,'mask.npy'))

## Part 5: Radiomics Use-case Example <a name="RADIOMICS"></a>

Here we use the popular open-source radiomics library PyRadiomics (https://pyradiomics.readthedocs.io/en/latest/) to calculate radiomic features for our ROIs. Results are consistent with 3D Slicer (https://www.slicer.org/) using Radiomics (https://www.slicer.org/wiki/Documentation/Nightly/Extensions/Radiomics) extension.  

In [ ]:
def convert_array(array, index):
    """
    Creates a masked array specific for a given ROI index. 
        Parameters:
            array (np array): Input array corresponding to all masks superimposed on image.
            index (int): Index to mask with respect to, i.e. only select cetain sub-mask.
        Returns:
            array_new (np array): Output masked array.
    """
    array_new = array.copy() # must create copy or else changes original 
    array_new[array_new!=index] = 0
    array_new[array_new==index] = 1
    return array_new

In [ ]:
%%time
mask_new = convert_array(mask,3) # only consider mask for GTV (index 3)

sitk_img = sitk.GetImageFromArray(image) # need sitk image to plug into PyRadiomics
sitk_mask = sitk.GetImageFromArray(mask_new) 

params = {} # can edit in more params as neccessary 
extractor = featureextractor.RadiomicsFeatureExtractor(**params) # instantiate extractor with parameters 
extractor.disableAllFeatures() # in case where only want some features, can delete these lines if you want deafult
extractor.enableFeatureClassByName('firstorder') 
extractor.enableFeatureClassByName('glcm') 
features = {} # empty dictionary 
features = extractor.execute(sitk_img, sitk_mask) # unpack results into features dictionary
df = pd.DataFrame({k: [v] for k, v in features.items()}) # put dictionary into a dataframe 

In [ ]:
df